# Dự án NLP: Phân loại Văn bản Tiếng Việt

Notebook này triển khai quy trình hoàn chỉnh để phân loại tin tức tiếng Việt thành 20 chủ đề khác nhau.

**Quy trình thực hiện:**
1.  **Tải & Tiền xử lý dữ liệu:** Chuẩn hóa Unicode, tách từ (tokenization), loại bỏ từ dừng (stopwords).
2.  **Mô hình Machine Learning cơ bản:** Naive Bayes, Logistic Regression, SVM (LinearSVC), SGD Classifier.
3.  **Mô hình Deep Learning:** LSTM (Long Short-Term Memory).
4.  **Mô hình Transformers:** Fine-tuning PhoBERT (Sử dụng chiến thuật cắt ghép đầu-đuôi).
5.  **Đánh giá & Dự đoán:** Xuất báo cáo hiệu năng và hệ thống dự đoán thời gian thực.

In [ ]:
# 1. THIẾT LẬP MÔI TRƯỜNG & THƯ VIỆN
import os
import gc
import joblib
import shutil
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
from tqdm import tqdm

# PyTorch & Transformers
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW

# Scikit-Learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.calibration import CalibratedClassifierCV

# Thư viện xử lý tiếng Việt
from pyvi import ViTokenizer

# Cấu hình đường dẫn
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR if (CURRENT_DIR / "data").exists() else CURRENT_DIR.parent

DATA_DIR = PROJECT_ROOT / "data" / "final"
MODEL_DIR = PROJECT_ROOT / "models"
REPORT_DIR = PROJECT_ROOT / "reports"
JSONL_PATH = DATA_DIR / "nlp_dataset.jsonl"

# Tạo thư mục nếu chưa tồn tại
for d in [MODEL_DIR, REPORT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Thiết bị đang sử dụng: {DEVICE}")

Thiết bị đang sử dụng: cuda


In [2]:
# 2. TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU

# Danh sách từ dừng (Stopwords)
STOPWORDS = {
    "thì", "là", "mà", "của", "những", "các", "để", "và", "với", "có", 
    "trong", "đã", "đang", "sẽ", "được", "bị", "tại", "vì", "như", "này",
    "cho", "về", "một", "người", "khi", "ra", "vào", "lên", "xuống",
    "tôi", "chúng_tôi", "bạn", "họ", "chúng_ta", "theo", "ông", "bà",
    "nhiều", "ít", "rất", "quá", "lắm", "nhưng", "tuy_nhiên", "nếu", "dù",
    "bài", "viết", "ảnh", "video", "clip", "nguồn"
}

def normalize_text(text):
    """Chuẩn hóa Unicode về dạng NFC."""
    return unicodedata.normalize('NFC', text)

def preprocess_text(text):
    """Chuẩn hóa, tách từ và loại bỏ từ dừng."""
    text = normalize_text(text)
    tokenized = ViTokenizer.tokenize(text)
    words = tokenized.split()
    clean_words = [w for w in words if w.lower() not in STOPWORDS]
    return " ".join(clean_words)

if JSONL_PATH.exists():
    print("Đang tải dữ liệu từ file JSONL...")
    df = pd.read_json(JSONL_PATH, lines=True)
    
    # Đảm bảo có cột raw_text (cho PhoBERT)
    if 'raw_text' not in df.columns:
        print("Đang tạo cột 'raw_text'...")
        df['raw_text'] = df['text'].apply(normalize_text)
    
    # Đảm bảo cột text đã được xử lý (cho ML/LSTM)
    print("Đang xử lý tách từ và lọc stopwords...")
    tqdm.pandas(desc="Xử lý văn bản")
    df['text'] = df['raw_text'].progress_apply(preprocess_text)
    
    # Lưu lại file đã xử lý để lần sau dùng nhanh hơn
    df.to_json(JSONL_PATH, orient="records", lines=True)
else:
    print("Lỗi: Không tìm thấy file dữ liệu. Vui lòng kiểm tra lại thư mục data.")

# Mã hóa nhãn (Label Encoding)
target_col = 'label_name'
if target_col not in df.columns and 'label' in df.columns:
    df[target_col] = df['label']

le = LabelEncoder()
df['label_id'] = le.fit_transform(df[target_col])
classes = le.classes_
num_classes = len(classes)

# Chia tập Train/Test (Tỷ lệ 80/20)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[target_col])

print(f"Số lượng Train: {len(train_df)} | Số lượng Test: {len(test_df)}")
print(f"Danh sách {num_classes} nhãn: {classes}")

del df
gc.collect()

Đang tải dữ liệu từ file JSONL...
Đang xử lý tách từ và lọc stopwords...


Xử lý văn bản: 100%|██████████| 115188/115188 [09:09<00:00, 209.46it/s]


Số lượng Train: 92150 | Số lượng Test: 23038
Danh sách 20 nhãn: ['Bất động sản' 'Chứng khoán' 'Công nghệ' 'Du lịch' 'Gia đình'
 'Giao thông' 'Giáo dục' 'Giải trí' 'Khoa học' 'Khởi nghiệp' 'Kinh doanh'
 'Nông nghiệp' 'Pháp luật' 'Sức khỏe' 'Thế giới' 'Thể thao'
 'Thời sự – Chính trị' 'Văn hóa' 'Đời sống' 'Ẩm thực']


0

# 3. MÔ HÌNH MACHINE LEARNING CƠ BẢN

In [ ]:
# Tạo đặc trưng TF-IDF
print("Đang tạo vector TF-IDF...")
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
X_train = tfidf.fit_transform(train_df['text'])
X_test = tfidf.transform(test_df['text'])

# 1. Naive Bayes
print("Đang huấn luyện Naive Bayes...")
nb = MultinomialNB()
nb.fit(X_train, train_df['label_id'])
acc_nb = accuracy_score(test_df['label_id'], nb.predict(X_test))
print(f"Naive Bayes Accuracy: {acc_nb:.4f}")

# 2. Logistic Regression
print("Đang huấn luyện Logistic Regression...")
lr = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
lr.fit(X_train, train_df['label_id'])
acc_lr = accuracy_score(test_df['label_id'], lr.predict(X_test))
print(f"Logistic Regression Accuracy: {acc_lr:.4f}")

# 3. SVM (LinearSVC) - NÂNG CẤP CALIBRATION
print("Đang huấn luyện SVM (LinearSVC) với chế độ chuẩn hóa xác suất...")
linear_svc = LinearSVC(dual=False, random_state=42, max_iter=1000)
svm = CalibratedClassifierCV(linear_svc, method='sigmoid', cv=5) 
svm.fit(X_train, train_df['label_id'])
acc_svm = accuracy_score(test_df['label_id'], svm.predict(X_test))
print(f"SVM (Calibrated) Accuracy: {acc_svm:.4f}")

# 4. SGD Classifier
print("Đang huấn luyện SGD Classifier...")
sgd = SGDClassifier(loss='modified_huber', penalty='l2', alpha=1e-4, 
                    random_state=42, max_iter=1000, tol=1e-3, n_jobs=-1)
sgd.fit(X_train, train_df['label_id'])
acc_sgd = accuracy_score(test_df['label_id'], sgd.predict(X_test))
print(f"SGD Classifier Accuracy: {acc_sgd:.4f}")

Đang tạo vector TF-IDF...
Đang huấn luyện Naive Bayes...
Naive Bayes Accuracy: 0.8235
Đang huấn luyện Logistic Regression...
Logistic Regression Accuracy: 0.8801
Đang huấn luyện SVM (LinearSVC)...
SVM (LinearSVC) Accuracy: 0.8863
Đang huấn luyện SGD Classifier...
SGD Classifier Accuracy: 0.8808


# 4. DEEP LEARNING (LSTM)

In [4]:
print("Đang huấn luyện mô hình LSTM...")

# Xây dựng bộ từ vựng (Vocabulary)
counter = Counter()
for t in train_df['text']: 
    counter.update(t.split())

vocab = {w: i+2 for i, (w, _) in enumerate(counter.most_common(20000))}
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1
MAX_LEN_LSTM = 1024

def text_to_seq(text, vocab, max_len):
    seq = [vocab.get(w, 1) for w in text.split()]
    if len(seq) < max_len:
        seq += [0] * (max_len - len(seq))
    return seq[:max_len]

class LSTMDataset(Dataset):
    def __init__(self, df): 
        self.x = [text_to_seq(t, vocab, MAX_LEN_LSTM) for t in df['text']]
        self.y = df['label_id'].values
    def __len__(self): 
        return len(self.y)
    def __getitem__(self, idx): 
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])

train_loader = DataLoader(LSTMDataset(train_df), batch_size=64, shuffle=True)
test_loader = DataLoader(LSTMDataset(test_df), batch_size=64)

# Kiến trúc mô hình LSTM
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        embedded = self.embedding(x)
        output, (h_n, c_n) = self.lstm(embedded)
        last_hidden = h_n[-1]
        out = self.fc(last_hidden)
        return out

model_lstm = LSTMClassifier(len(vocab)+2, 128, 128, num_classes).to(DEVICE)
optimizer = optim.Adam(model_lstm.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Vòng lặp huấn luyện
for epoch in range(10):
    model_lstm.train()
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model_lstm(x), y)
        loss.backward()
        optimizer.step()

# Đánh giá
model_lstm.eval()
preds_lstm = []
with torch.no_grad():
    for x, _ in test_loader:
        preds_lstm.extend(torch.argmax(model_lstm(x.to(DEVICE)), dim=1).cpu().numpy())

acc_lstm = accuracy_score(test_df['label_id'], preds_lstm)
print(f"LSTM Accuracy: {acc_lstm:.4f}")

Đang huấn luyện mô hình LSTM...


c:\Users\ASUS\anaconda3\envs\trading\Lib\site-packages\torch\nn\modules\module.py:1371: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:39.)
  return t.to(
Epoch 10: 100%|██████████| 1440/1440 [00:15<00:00, 93.71it/s]


LSTM Accuracy: 0.8192


# 5. TRANSFORMERS (PHOBERT)

In [5]:
# print("Đang chuẩn bị huấn luyện PhoBERT...")

# # Tham số cấu hình
# MAX_LEN_BERT = 256
# BATCH_SIZE = 32
# LEARNING_RATE = 2e-5
# EPOCHS = 5
# MODEL_NAME = "vinai/phobert-base-v2"

# PHOBERT_DIR = MODEL_DIR / "phobert_best"
# PHOBERT_DIR.mkdir(parents=True, exist_ok=True)

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# class PhoBERTDataset(Dataset):
#     def __init__(self, df, tokenizer, max_len):
#         self.texts = df['raw_text'].tolist()
#         self.labels = df['label_id'].tolist()
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         text = str(self.texts[idx])
#         tokens = self.tokenizer.encode(text, add_special_tokens=True)
        
#         # Chiến thuật cắt ghép Head + Tail để lấy được thông tin đầu và cuối bài viết
#         if len(tokens) > self.max_len:
#             head_len = 128
#             tail_len = self.max_len - head_len
#             input_ids = tokens[:head_len] + tokens[-tail_len:]
#         else:
#             padding_len = self.max_len - len(tokens)
#             input_ids = tokens + [self.tokenizer.pad_token_id] * padding_len
            
#         input_ids = torch.tensor(input_ids)
#         attention_mask = (input_ids != self.tokenizer.pad_token_id).long()
        
#         return {
#             'input_ids': input_ids,
#             'attention_mask': attention_mask,
#             'labels': torch.tensor(self.labels[idx])
#         }

# train_loader_bert = DataLoader(PhoBERTDataset(train_df, tokenizer, MAX_LEN_BERT), batch_size=BATCH_SIZE, shuffle=True)
# test_loader_bert = DataLoader(PhoBERTDataset(test_df, tokenizer, MAX_LEN_BERT), batch_size=BATCH_SIZE)

# model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_classes).to(DEVICE)
# optimizer = AdamW(model_bert.parameters(), lr=LEARNING_RATE)

# best_acc = 0.0

# for epoch in range(EPOCHS):
#     print(f"Epoch {epoch+1}/{EPOCHS}")
#     model_bert.train()
#     train_bar = tqdm(train_loader_bert, desc="Training")
    
#     for batch in train_bar:
#         input_ids = batch['input_ids'].to(DEVICE)
#         attention_mask = batch['attention_mask'].to(DEVICE)
#         labels = batch['labels'].to(DEVICE)
        
#         optimizer.zero_grad()
#         outputs = model_bert(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         train_bar.set_postfix({'loss': f"{loss.item():.4f}"})
    
#     # Đánh giá
#     model_bert.eval()
#     preds, targets = [], []
#     with torch.no_grad():
#         for batch in tqdm(test_loader_bert, desc="Evaluating"):
#             outputs = model_bert(batch['input_ids'].to(DEVICE), attention_mask=batch['attention_mask'].to(DEVICE))
#             preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
#             targets.extend(batch['labels'].numpy())
    
#     acc = accuracy_score(targets, preds)
#     print(f"Validation Accuracy: {acc:.4f}")
    
#     if acc > best_acc:
#         print(f"Kỷ lục mới (Acc: {acc:.4f}). Đang lưu model...")
#         model_bert.save_pretrained(PHOBERT_DIR)
#         tokenizer.save_pretrained(PHOBERT_DIR)
#         best_acc = acc

# print(f"Độ chính xác tốt nhất của PhoBERT: {best_acc:.4f}")

# 6. ĐÁNH GIÁ & BÁO CÁO

In [6]:
# 1. Lưu trữ tất cả model
print("Đang lưu các mô hình...")
joblib.dump(le, MODEL_DIR / "label_encoder.pkl")
joblib.dump(tfidf, MODEL_DIR / "tfidf_vectorizer.pkl")
joblib.dump(nb, MODEL_DIR / "naive_bayes.pkl")
joblib.dump(lr, MODEL_DIR / "logistic_regression.pkl")
joblib.dump(svm, MODEL_DIR / "svm_linear.pkl")
joblib.dump(sgd, MODEL_DIR / "sgd_classifier.pkl")

lstm_checkpoint = {
    'vocab': vocab, 
    'model_state': model_lstm.state_dict(),
    'config': {
        'vocab_size': len(vocab)+2, 
        'embed_dim': 100, 
        'hidden_dim': 100, 
        'num_classes': num_classes, 
        'max_len': MAX_LEN_LSTM
    }
}
torch.save(lstm_checkpoint, MODEL_DIR / "lstm_model.pth")
print("Đã lưu thành công tất cả model.")

# 2. Tạo bảng tổng hợp kết quả
results_df = pd.DataFrame([
    {"Model": "SVM (LinearSVC)", "Accuracy": acc_svm},
    {"Model": "SGD Classifier", "Accuracy": acc_sgd},
    {"Model": "Logistic Regression", "Accuracy": acc_lr},
    {"Model": "Naive Bayes", "Accuracy": acc_nb},
    {"Model": "LSTM", "Accuracy": acc_lstm},
    # {"Model": "PhoBERT (Best)", "Accuracy": best_acc}
]).sort_values(by="Accuracy", ascending=False)

print("\nBảng Xếp Hạng Hiệu Năng:")
display(results_df)
results_df.to_excel(REPORT_DIR / "final_leaderboard.xlsx", index=False)

Đang lưu các mô hình...
Đã lưu thành công tất cả model.

Bảng Xếp Hạng Hiệu Năng:


,Model,Accuracy
0,SVM (LinearSVC),0.886275
1,SGD Classifier,0.880806
2,Logistic Regression,0.880111
3,Naive Bayes,0.823509
4,LSTM,0.819212


# 7. HỆ THỐNG DỰ ĐOÁN THỰC TẾ (INFERENCE)

In [ ]:
# ==============================================================================
# 7. HỆ THỐNG DỰ ĐOÁN (FINAL POLISH - FULL PROBABILITY)
# ==============================================================================
import torch
import torch.nn as nn
import joblib
import requests
import numpy as np # Cần numpy để tính Softmax cho SVM
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path
from pyvi import ViTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- CẤU HÌNH ---
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR if (CURRENT_DIR / "data").exists() else CURRENT_DIR.parent
MODEL_DIR = PROJECT_ROOT / "models"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Device: {DEVICE}")

# --- LSTM CLASS ---
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        embedded = self.embedding(x)
        output, (h_n, c_n) = self.lstm(embedded)
        return self.fc(h_n[-1])

# --- LOAD MODELS ---
le, tfidf = None, None
nb, lr, svm, sgd = None, None, None, None
lstm_model, lstm_vocab, lstm_config = None, {}, {}
model_bert, tokenizer_bert = None, None

try:
    # 1. Sklearn
    if (MODEL_DIR / "label_encoder.pkl").exists(): le = joblib.load(MODEL_DIR / "label_encoder.pkl")
    if (MODEL_DIR / "tfidf_vectorizer.pkl").exists(): tfidf = joblib.load(MODEL_DIR / "tfidf_vectorizer.pkl")
    if (MODEL_DIR / "naive_bayes.pkl").exists(): nb = joblib.load(MODEL_DIR / "naive_bayes.pkl")
    if (MODEL_DIR / "logistic_regression.pkl").exists(): lr = joblib.load(MODEL_DIR / "logistic_regression.pkl")
    if (MODEL_DIR / "svm_linear.pkl").exists(): svm = joblib.load(MODEL_DIR / "svm_linear.pkl")
    if (MODEL_DIR / "sgd_classifier.pkl").exists(): sgd = joblib.load(MODEL_DIR / "sgd_classifier.pkl")

    # 2. LSTM (FIX SIZE 128)
    if (MODEL_DIR / "lstm_model.pth").exists():
        ckpt = torch.load(MODEL_DIR / "lstm_model.pth", map_location='cpu')
        lstm_config, lstm_vocab = ckpt['config'], ckpt['vocab']
        
        # Hardcode 128 để khớp với file đã train
        lstm_model = LSTMClassifier(
            lstm_config.get('vocab_size', len(lstm_vocab) + 2),
            128, 128, 
            lstm_config.get('num_classes', 20)
        )
        lstm_model.load_state_dict(ckpt['model_state'])
        lstm_model.to(DEVICE).eval()

    # 3. PhoBERT
    PHOBERT_PATH = MODEL_DIR / "phobert_best"
    if PHOBERT_PATH.exists():
        tokenizer_bert = AutoTokenizer.from_pretrained(PHOBERT_PATH)
        model_bert = AutoModelForSequenceClassification.from_pretrained(PHOBERT_PATH).to(DEVICE)
        model_bert.eval()
    elif 'model_bert' in globals() and model_bert:
        tokenizer_bert = globals().get('tokenizer')

except Exception as e:
    print(f"Load Error: {e}")

# --- PREDICT FUNCTION ---
def predict_all_models(url_or_text):
    print(f"\nInput: {url_or_text[:60]}...")
    
    # Get Content
    content = url_or_text
    if url_or_text.startswith("http"):
        try:
            headers = {'User-Agent': 'Mozilla/5.0'}
            resp = requests.get(url_or_text, headers=headers, timeout=10)
            soup = BeautifulSoup(resp.content, 'html.parser')
            content = ' '.join([p.get_text() for p in soup.find_all('p')])
            if len(content) < 50: return print("Short content.")
        except: return print("URL Error.")

    # Preprocess
    text_seg = ViTokenizer.tokenize(content)
    
    print("-" * 65)
    print(f"{'MODEL':<20} | {'LABEL':<30} | {'CONF'}")
    print("-" * 65)

    # 1. ML Predict
    if tfidf and le:
        vec = tfidf.transform([text_seg])
        
        # Naive Bayes
        if nb:
            print(f"{'Naive Bayes':<20} | {le.inverse_transform(nb.predict(vec))[0].upper():<30} | {nb.predict_proba(vec).max():.2%}")
        
        # Logistic Regression
        if lr:
            print(f"{'Logistic Reg':<20} | {le.inverse_transform(lr.predict(vec))[0].upper():<30} | {lr.predict_proba(vec).max():.2%}")
        
        # SVM (LinearSVC)
        if svm:
            pred = svm.predict(vec)[0]
            l = le.inverse_transform([pred])[0]
            try:
                dec = svm.decision_function(vec) 
                probs = np.exp(dec) / np.sum(np.exp(dec), axis=1, keepdims=True)
                p = f"{probs.max():.2%}"
            except:
                p = "N/A"
            print(f"{'SVM':<20} | {l.upper():<30} | {p}")

        # SGD
        if sgd:
            p = f"{sgd.predict_proba(vec).max():.2%}" if hasattr(sgd, "predict_proba") else "N/A"
            print(f"{'SGD':<20} | {le.inverse_transform(sgd.predict(vec))[0].upper():<30} | {p}")

    # 2. LSTM Predict
    if lstm_model and lstm_vocab:
        max_len = lstm_config.get('max_len', 1024)
        seq = [lstm_vocab.get(w, 1) for w in text_seg.split()]
        seq = (seq + [0]*(max_len-len(seq)))[:max_len]
        
        model_device = next(lstm_model.parameters()).device
        seq_tensor = torch.tensor([seq], dtype=torch.long).to(model_device)
        
        with torch.no_grad():
            out = lstm_model(seq_tensor)
            prob, idx = torch.max(torch.softmax(out, dim=1), dim=1)
            print(f"{'LSTM':<20} | {le.inverse_transform([idx.item()])[0].upper():<30} | {prob.item():.2%}")

    # 3. PhoBERT Predict
    if model_bert and tokenizer_bert:
        tokens = tokenizer_bert.encode(content, add_special_tokens=True)
        ids = (tokens[:128] + tokens[-128:]) if len(tokens) > 256 else (tokens + [tokenizer_bert.pad_token_id]*(256-len(tokens)))
        
        model_device = next(model_bert.parameters()).device
        ids_t = torch.tensor([ids]).to(model_device)
        mask_t = (ids_t != tokenizer_bert.pad_token_id).long().to(model_device)
        
        with torch.no_grad():
            out = model_bert(ids_t, attention_mask=mask_t)
            prob, idx = torch.max(torch.softmax(out.logits, dim=1), dim=1)
            print(f"{'PhoBERT':<20} | {le.inverse_transform([idx.item()])[0].upper():<30} | {prob.item():.2%}")
    
    print("-" * 65)

# --- TEST ---
link_test = "https://vnexpress.net/new-york-cai-cach-nha-tu-sau-vu-quan-giao-danh-chet-pham-nhan-4996107.html"
predict_all_models(link_test)

Device: cuda

Input: https://vnexpress.net/new-york-cai-cach-nha-tu-sau-vu-quan-g...
-----------------------------------------------------------------
MODEL                | LABEL                          | CONF
-----------------------------------------------------------------
Naive Bayes          | THẾ GIỚI                       | 99.38%
Logistic Reg         | THẾ GIỚI                       | 81.42%
SVM                  | THẾ GIỚI                       | 27.90%
SGD                  | THẾ GIỚI                       | 88.51%
LSTM                 | THẾ GIỚI                       | 98.17%
PhoBERT              | THẾ GIỚI                       | 99.40%
-----------------------------------------------------------------
